# API 
2025 labs done in Codespaces/Github

In [79]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Data scientist Job

In [ ]:
import requests
import time

def get_vacancies():
    url = 'https://api.hh.ru/vacancies'
    jsons_list = []
    
    headers = {'User-Agent': 'TestBot'}
    
    # Data Scientist с хорошей зарплатой, удалённо
    params = {
        'text': 'Data Scientist',
        'per_page': '100',
        'page': 0,
        'only_with_salary': True,
        'schedule': 'remote',
        'salary': 150000,
        'currency': 'RUR',
        'search_field': 'name',
        'order_by': 'salary_desc',
        'experience': 'between3And6',
    }
    

    for num_page in range(5):
        params['page'] = num_page
        
        try:
            src = requests.get(url, params=params, headers=headers, timeout=10)
            
            if src.status_code != 200:
                print(f"Страница {num_page}: Ошибка {src.status_code}")
                break
                
            data = src.json()
            
            if 'items' not in data or not data['items']:
                print(f"Страница {num_page}: больше нет вакансий")
                break
                
            jsons_list.append(data)
            print(f"Страница {num_page+1}: {len(data['items'])} вакансий")
            
            # pause api to fix timeout
            time.sleep(0.3)
            
        except requests.exceptions.Timeout:
            print(f"Страница {num_page}: Таймаут, пропуск")
            continue
        except Exception as e:
            print(f"Страница {num_page}: Ошибка {type(e).__name__}")
            break
    
    print(f"\nСобрано: {len(jsons_list)} страниц с вакансиями")
    if jsons_list:
        total_vacancies = sum(len(page['items']) for page in jsons_list)
        print(f"Всего вакансий: {total_vacancies}")
        
        # gold_job
        if jsons_list[0]['items']:
            top_vacancy = jsons_list[0]['items'][0]
            salary = top_vacancy.get('salary', {})
            print(f"gold_job: {top_vacancy['name']}")
            print(f"   coin: {salary.get('from', '?')} - {salary.get('to', '?')} {salary.get('currency', 'руб.')}")
            if 'schedule' in top_vacancy:
                print(f"   График: {top_vacancy['schedule'].get('name', '?')}")
    
    return jsons_list


In [101]:
vacancies = get_vacancies()

Страница 1: 10 вакансий
Страница 1: больше нет вакансий

Собрано: 1 страниц с вакансиями
Всего вакансий: 10
gold_job: Lead Data Scientist (LLM) / Ведущий специалист по данным
   coin: None - 5000 USD
   График: Удаленная работа


# Job Table

In [102]:
import pandas as pd

data = pd.DataFrame(columns=[
    'title',               # Название
    'salary_min',          # ЗП от
    'salary_max',          # ЗП до  
    'experience',          # Опыт 3-6
    'company',             # Компания
    'location',            # Локация
    'work_schedule',       # График = удаленка
    'remote_type',         # Формат
    'key_requirements',    # Ключевые требования
    'is_trusted_employer', # Проверенный работодатель
    'has_test',            # Тест!
    'url'                  # Ссылка
])

for col in data.columns:
    print(f"  -> {col}")

  -> title
  -> salary_min
  -> salary_max
  -> experience
  -> company
  -> location
  -> work_schedule
  -> remote_type
  -> key_requirements
  -> is_trusted_employer
  -> has_test
  -> url


In [103]:
for page in vacancies:
    if 'items' not in page:
        continue
        
    for vacancy in page['items']:
        row = {
            'title': vacancy.get('name', ''),
            'url': vacancy.get('alternate_url', ''),
        }
        
        salary_data = vacancy.get('salary')
        if salary_data:
            row['salary_min'] = salary_data.get('from')
            row['salary_max'] = salary_data.get('to')
        else:
            row['salary_min'] = None
            row['salary_max'] = None
        
        experience_data = vacancy.get('experience', {})
        row['experience'] = experience_data.get('name', '') if experience_data else ''
        
        employer_data = vacancy.get('employer', {})
        row['company'] = employer_data.get('name', '')
        row['is_trusted_employer'] = employer_data.get('trusted', False)
        
        area_data = vacancy.get('area', {})
        row['location'] = area_data.get('name', '')
        
        schedule_data = vacancy.get('schedule', {})
        row['work_schedule'] = schedule_data.get('name', '')
        
        work_format_data = vacancy.get('work_format', [{}])
        row['remote_type'] = work_format_data[0].get('name', '') if work_format_data else ''
        
        snippet_data = vacancy.get('snippet', {})
        requirements = snippet_data.get('requirement', '')
        row['key_requirements'] = requirements[:150] if requirements else ''
        
        row['has_test'] = vacancy.get('has_test', False)
        
        new_row_df = pd.DataFrame([row])
        data = pd.concat([data, new_row_df], ignore_index=True)

data

,title,salary_min,salary_max,experience,company,location,work_schedule,remote_type,key_requirements,is_trusted_employer,has_test,url
0,Lead Data Scientist (LLM) / Ведущий специалист...,None,5000,От 3 до 6 лет,WaveAccess,Санкт-Петербург,Удаленная работа,Удалённо,5+ years of commercial experience in <highligh...,True,False,https://hh.ru/vacancy/128483349
1,Senior ML / Data Scientist,3500,4350,От 3 до 6 лет,ПБК Менеджмент,Минск,Удаленная работа,Удалённо,Мы усиливаем команду разработки моделей и ищем...,True,False,https://hh.ru/vacancy/128152732
2,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Москва,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/127981401
3,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Казань,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913274
4,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Санкт-Петербург,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913275
5,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Екатеринбург,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913276
6,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Нижний Новгород,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913277
7,Data Scientist,None,300000,От 3 до 6 лет,Детский мир,Москва,Удаленная работа,На месте работодателя,Уверенное владение знаниями о работе и создани...,True,True,https://hh.ru/vacancy/127795453
8,Data analyst / Data scientist,230000,250000,От 3 до 6 лет,ROSSKO,Москва,Удаленная работа,Удалённо,Вы умеете выявлять факторы влияния на эффектив...,True,False,https://hh.ru/vacancy/127307468
9,Data Scientist,None,180000,От 3 до 6 лет,Hawking Bros,Москва,Удаленная работа,Удалённо,Опыт работы с табличными данными и классически...,True,False,https://hh.ru/vacancy/128102483


In [104]:
data.head(1)

,title,salary_min,salary_max,experience,company,location,work_schedule,remote_type,key_requirements,is_trusted_employer,has_test,url
0,Lead Data Scientist (LLM) / Ведущий специалист...,None,5000,От 3 до 6 лет,WaveAccess,Санкт-Петербург,Удаленная работа,Удалённо,5+ years of commercial experience in <highligh...,True,False,https://hh.ru/vacancy/128483349


Lead Data Scientist (LLM) / Ведущий специалист по данным

до 5 000 $ за месяц

Формат работы: удалённо

In [105]:
data.iloc[0]['key_requirements']

'5+ years of commercial experience in <highlighttext>data</highlighttext> science or ML engineering. Ability to design the overall approach to solving.'

Виджет по api.hh.ru

In [1]:
from IPython.display import IFrame

# url
widget_url = "https://api.hh.ru/widgets/vacancies/employer?employer_id=2251053&locale=RU&links_color=1560b2&border_color=1560b2&title=&show_region=true&professional_role=165"

# widget
IFrame(src=widget_url, width=350, height=250)